In [1]:
import looker_sdk
import os
import json
base_url= "https://rushstreet.looker.com:19999"
# API 3 client id
client_id="REDACTED"
# API 3 client secret
client_secret="REDACTED"
# Set to false if testing locally against self-signed certs. Otherwise leave True
verify_ssl= "true"
os.environ["LOOKERSDK_BASE_URL"] = base_url #If your looker URL has .cloud in it (hosted on GCP), do not include :19999 (ie: https://your.cloud.looker.com).
os.environ["LOOKERSDK_API_VERSION"] = "4.0" #3.1 is the default version. You can change this to 4.0 if you want.
os.environ["LOOKERSDK_VERIFY_SSL"] = "true" #Defaults to true if not set. SSL verification should generally be on unless you have a real good reason not to use it. Valid options: true, y, t, yes, 1.
os.environ["LOOKERSDK_TIMEOUT"] = "120" #Seconds till request timeout. Standard default is 120.

#Get the following values from your Users page in the Admin panel of your Looker instance > Users > Your user > Edit API keys. If you know your user id, you can visit https://your.looker.com/admin/users/<your_user_id>/edit.
os.environ["LOOKERSDK_CLIENT_ID"] =  client_id #No defaults.
os.environ["LOOKERSDK_CLIENT_SECRET"] = client_secret #No defaults. This should be protected at all costs. Please do not leave it sitting here, even if you don't share this document.

print("All environment variables set.")
sdk = looker_sdk.init40()
print('Looker SDK 4.0 initialized successfully.')



All environment variables set.
Looker SDK 4.0 initialized successfully.


In [2]:
#@title Create User list with email/user_id values
user_list = dict()
users = sdk.all_users()
for user in users:
  if user['email'] and not user['is_disabled']: #removing users that don't have emails setup and aren't disabled
    user_list[user['email']] = user['id']
num_users = len(user_list)

In [3]:
#@title Create small function to retrieve names and content_types of favorited content

def identify_content(id):
  response = sdk.content_metadata(content_metadata_id=id)
  return response['name'],response['content_type']

In [4]:
#@title Check all content if the current user has favorited them or not
favorited_content = dict()
def retrieve_favorite_content(user_id):
  response = sdk.search_content_favorites(user_id=user_id)
  for r in response:
    name, content_type = identify_content(r['content_metadata_id']) 
    email = [k for k,v in user_list.items() if v == user_id][0] #Change user_id into email for data readability
    if email in favorited_content: #If we already have a favorited content, we need to append the new content
      if (content_type,name) in favorited_content[email]:
        continue
      else:
        favorited_content[email].append((content_type,name))
    else: #If we don't have any favorited content yet, we add the new content as a list
      favorited_content[email] = []
      favorited_content[email].append((content_type,name))



In [5]:
#@title Loop through all users and query their favorited content and save to the favorited_content dictionary
for index,(k,v) in enumerate(user_list.items()):
  index = index+1
  users_left = num_users - index
  print(f"Querying the #{index} user in the list, {users_left} users left.")
  sdk.auth.logout() #This is required if you stopped the sdk mid-way
  sudo_auth = sdk.login_user(v)
  sdk.auth.login_user(v)
  user_id=sdk.me()["id"]
  retrieve_favorite_content(user_id)
print(favorited_content)

Querying the #1 user in the list, 309 users left.


C:\Users\JamesChu\anaconda3\lib\site-packages\looker_sdk\sdk\api40\methods.py:419: UserWarning: login_user behavior changed significantly in 21.4.0. See https://git.io/JOtH1
  warnings.warn("login_user behavior changed significantly in 21.4.0. See https://git.io/JOtH1")


Querying the #2 user in the list, 308 users left.
Querying the #3 user in the list, 307 users left.
Querying the #4 user in the list, 306 users left.
Querying the #5 user in the list, 305 users left.
Querying the #6 user in the list, 304 users left.
Querying the #7 user in the list, 303 users left.
Querying the #8 user in the list, 302 users left.
Querying the #9 user in the list, 301 users left.
Querying the #10 user in the list, 300 users left.
Querying the #11 user in the list, 299 users left.
Querying the #12 user in the list, 298 users left.
Querying the #13 user in the list, 297 users left.
Querying the #14 user in the list, 296 users left.
Querying the #15 user in the list, 295 users left.
Querying the #16 user in the list, 294 users left.
Querying the #17 user in the list, 293 users left.
Querying the #18 user in the list, 292 users left.
Querying the #19 user in the list, 291 users left.
Querying the #20 user in the list, 290 users left.
Querying the #21 user in the list, 289 

Querying the #162 user in the list, 148 users left.
Querying the #163 user in the list, 147 users left.
Querying the #164 user in the list, 146 users left.
Querying the #165 user in the list, 145 users left.
Querying the #166 user in the list, 144 users left.
Querying the #167 user in the list, 143 users left.
Querying the #168 user in the list, 142 users left.
Querying the #169 user in the list, 141 users left.
Querying the #170 user in the list, 140 users left.
Querying the #171 user in the list, 139 users left.
Querying the #172 user in the list, 138 users left.
Querying the #173 user in the list, 137 users left.
Querying the #174 user in the list, 136 users left.
Querying the #175 user in the list, 135 users left.
Querying the #176 user in the list, 134 users left.
Querying the #177 user in the list, 133 users left.
Querying the #178 user in the list, 132 users left.
Querying the #179 user in the list, 131 users left.
Querying the #180 user in the list, 130 users left.
Querying the

In [6]:
#@title Loop through the favorited dictionary and separate each dictionary into dashboards/boards/looks

favorited_dashboards = dict()
favorited_boards = dict()
favorited_looks = dict()
for k,v in favorited_content.items():
  email = k
  for content in v:
    content_type = content[0]
    content_name = content[1]
    if content_type == "dashboard":
      if favorited_dashboards.get(email):
        favorited_dashboards[email].append(content_name)
      else:
        favorited_dashboards[email] = []
        favorited_dashboards[email].append(content_name)
    elif content_type == "homepage":
      if favorited_boards.get(email):
        favorited_boards[email].append(content_name)
      else:
        favorited_boards[email] = []
        favorited_boards[email].append(content_name)
    elif content_type == "look":
      if favorited_looks.get(email):
        favorited_looks[email].append(content_name)
      else:
        favorited_looks[email] = []
        favorited_looks[email].append(content_name)
print(favorited_dashboards)
print(favorited_boards)
print(favorited_looks)

{'adams@rushstreetinteractive.com': ['Daily Weekly Monthly', 'Pending Tokens'], 'alford-tate@rushstreetinteractive.com': ['VIP Loyalty Level Overview'], 'ayala@rushstreetinteractive.com': ['Progress Monitor II', 'Colombia Daily Casino Detailed', 'Veriff Withdrawal Dashboard (Unaudited)', 'Veriff Withdrawal Dashboard (Audited)', 'Withdrawal Review v3'], 'beard@rushstreetinteractive.com': ['Geolocation Review Dashboard', 'Patron Summary', 'Deposit Monitoring', 'Progress Monitor II', 'Cage Rule Monitoring', 'Withdrawal Data', 'Payment KPIs', 'VERIFF Overview', 'Player Geolocations', 'Cage Rules Document Requests', 'Failed Deposit/Withdrawal Requests'], 'bell@rushstreetinteractive.com': ['C4F 30D KPI Charts', 'SH KPI Charts', 'RG KPI Charts', 'PA KPI Charts', 'DP KPI Charts', 'NY KPI Charts'], 'birdsall@rushstreetinteractive.com': ['VIP Loyalty Level Overview'], 'bonello@rushstreetinteractive.com': ['All KPIs', 'Marketing Messages US', 'Marketing Affiliates Conversion Funnel', 'CRM Lists- 

In [7]:
print(favorited_dashboards)

{'adams@rushstreetinteractive.com': ['Daily Weekly Monthly', 'Pending Tokens'], 'alford-tate@rushstreetinteractive.com': ['VIP Loyalty Level Overview'], 'ayala@rushstreetinteractive.com': ['Progress Monitor II', 'Colombia Daily Casino Detailed', 'Veriff Withdrawal Dashboard (Unaudited)', 'Veriff Withdrawal Dashboard (Audited)', 'Withdrawal Review v3'], 'beard@rushstreetinteractive.com': ['Geolocation Review Dashboard', 'Patron Summary', 'Deposit Monitoring', 'Progress Monitor II', 'Cage Rule Monitoring', 'Withdrawal Data', 'Payment KPIs', 'VERIFF Overview', 'Player Geolocations', 'Cage Rules Document Requests', 'Failed Deposit/Withdrawal Requests'], 'bell@rushstreetinteractive.com': ['C4F 30D KPI Charts', 'SH KPI Charts', 'RG KPI Charts', 'PA KPI Charts', 'DP KPI Charts', 'NY KPI Charts'], 'birdsall@rushstreetinteractive.com': ['VIP Loyalty Level Overview'], 'bonello@rushstreetinteractive.com': ['All KPIs', 'Marketing Messages US', 'Marketing Affiliates Conversion Funnel', 'CRM Lists- 

In [8]:
print(favorited_boards)

{'allswang@rushstreetinteractive.com': ['House Specials', 'C4F', 'Sportsbook Data'], 'angeloni@rushstreetinteractive.com': ['Service Recovery'], 'bonello@rushstreetinteractive.com': ['Payment'], 'botti@rushstreetinteractive.com': ["Jess's Board"], 'bucher@rushstreetinteractive.com': ['Jaycee'], 'campbell@rushstreetinteractive.com': ['ON - BR'], 'stephen.castagna@sportech.net': ['Accounting'], 'cebidanes@rushstreetinteractive.com': ['jc board'], 'csik@rushstreetinteractive.com': ['Android Reviews'], 'silva@rushstreetinteractive.com': ['charles board test'], 'darwick@rushstreetinteractive.com': ['Sportsbook Data'], 'diaz@rushstreetinteractive.com': ['TestFP'], 'paul.dionne@sportech.net': ['PD Board'], 'escobar@rushstreetinteractive.com': ['Sports', 'Bonus BTR', 'CRM', 'CPA and Reg Performance'], 'fillmyer@rushstreetinteractive.com': ['Service Recovery'], 'foody@rushstreetinteractive.com': ['Associates data'], 'glass@rushstreetinteractive.com': ['Adam'], 'rgordon@rushst.com': ['Liability'

In [9]:
print(favorited_looks)

{'alford-tate@rushstreetinteractive.com': ['Loyalty Level Changes', 'SF MI VIP', 'SF NJ VIP', 'SF PA VIP'], 'angeloni@rushstreetinteractive.com': ['Multi-State Search'], 'ayala@rushstreetinteractive.com': ['Detailed Kambi Lookup (All States)', 'Combined audited document queue', 'Initial Deposit Refund (First deposit, no real play on account)', 'VIP Secondary E-mails', 'Document Queue All States II', 'Universal Player Search', 'Combined unaudited WD queue'], 'beard@rushstreetinteractive.com': ['Detailed Kambi Lookup (All States)', 'Universal Player Search', 'Universal Payments Search', 'Blacklister template'], 'dbernal@rushstreetinteractive.com': ['EY Weekly Cash Transactions', 'EY Monthly Electronic Transactions', 'EY Monthly Cash Transactions', 'Coljuegos EY Weekly Revenues'], 'birnstein@rushstreetinteractive.com': ['Shops_Conciliations'], 'botti@rushstreetinteractive.com': ['25K DGC Point Challenge', 'VIP IGT Point Challenge Leaderboard', 'VIP King Cash Leaderboard'], 'brennan@rushst

In [10]:
import csv

In [11]:
with open('test.csv', 'w') as f:
    for key in favorited_dashboards.keys():
        f.write("%s,%s\n"%(key,favorited_dashboards[key]))

In [13]:
with open('favorited_boards.csv', 'w') as f:
    for key in favorited_boards.keys():
        f.write("%s,%s\n"%(key,favorited_boards[key]))

In [14]:
with open('favorited_looks.csv', 'w') as f:
    for key in favorited_looks.keys():
        f.write("%s,%s\n"%(key,favorited_looks[key]))